In [ ]:
# A bit of setup
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Create color maps
cmap_list = ['orange', 'cyan', 'cornflowerblue']
cmap_bold = ['darkorange', 'c', 'darkblue']
cmap_light = ListedColormap(cmap_list)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## <!-- dom:TITLE: Homework 1, PHY 959 -->
<!-- dom:AUTHOR: [PHY 959: Machine Learning in Physics]-->


# PHY 905: Homework Set #2
Due: **February 6, 2025**



___
***


# Problem 1: Exploring Regularization

In this problem, we will exercise what we learned about regularization.  We will use two different data sets that allow us to see regularization at work with a linear regression problem.  You will be fitting a polynomial function to data that is not quite linear, and has enough noise to confuse our choice of polynomial function.  And the data will be sparse enough so that overfitting will be a real concern.

We have to pretend that we don't know the highest order of the polynomial in the data, so we need to allow for a potentially large polynomial order.  But if we use a high enough polynomial order, we'll overfit our data and lose the ability to predict new data.  The goal will be to learn to identify cases where we've overfit our data and to use regularization to reduce the overfitting.

The workflow for this problem will be to:

  1. Part 1a: generate testing and training data samples.
  2. Part 1b: create a design matrix with polynomial features.
  3. Part 1c: use the mean squared error loss to find the analytical solution for the optimal weights.
  4. Part 1d: perform a gradient descent fit to the data, changing regularization strength to find the optical choice.
  5. Part 1e: plot the parameters of your polynomial fit as a function of regularization strength
  6. Part 1f: plot the loss for each fit as a function of regularization strength, relative to your test sample.

There is a lot of "starter" code here that will not work out of the box.  You'll need to add your own code to fill in some gaps that are missing.  Ask Prof. Fisher if you get stuck.

## Part 1a:

In the next cell, you are given:

  1. A training data sample to use to fit your polynomial
  2. An independent testing data sample to evaluate the quality of your fit

Make a plot of these two samples.  I suggest to use the `MatPlotLib` scatterplot method on the same graph using two different colors.

In [ ]:
# Build and visualize the data

# Make some training data
nPts = 10
np.random.seed(119)
Xtrain = np.random.uniform(0,1,nPts)
ytrain = 0.1*Xtrain + 1*np.sin(Xtrain*15) 

# Make some testing data, 3x as many as training
# Here we will use the same functional form to 
# effectively expand the data sample
np.random.seed(122)
Xtest = np.random.uniform(0,1,nPts*3)
ytest = 0.1*Xtest + 1*np.sin(Xtest*15)

# Because we're going to be plotting lines, it will be very convenient if
# our data is ordered by the absyssa.
print("Before permutation\n",Xtrain)
permute = Xtrain.argsort()
Xtrain=Xtrain[permute]
ytrain=ytrain[permute]
print("After permutation\n",Xtrain)

permute = Xtest.argsort()
Xtest=Xtest[permute]
ytest=ytest[permute]


## Part 1b:

Create a design/feature matrix for your samples.  In previous examples, the design matrix contained one variable for each dimension. Because we're now doing polynomial regression in one dimension, you will need to create a d-dimensional design matrix where d is the highest polynomial order.  For this problem, use d=7.  You can try out other values, but when you turn in your work please use 7 degrees of freedom.  Note that one of these will be the offset (bias) variable.

I strongly encourage you to try out the module ```PolynomialFeatures``` from ```scikit-learn```, which you have seen in our lecture discussion of January 12th.  Note that the variable to be transformed by the ```PolynomialFeatures``` `fit_transform` function needs to be an array of size ```(Npts,1)```.  Print out your design matrix so you can see what the ```PolynomialFeatures``` module does for you.  Don't forget to build a design matrix for your testing data!

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

nDegr = 7


## Part 1c:

Using your skills learned from homework #1, use the Numpy ```linalg``` module to calculate the vector of weights that best fit your training data.  Make a plot of your training data and your best fit line.  A convenient way to generate your line would be like this:  ```line = np.dot(DM,betas)``` wherein ```betas``` is the weights vector from your ```linalg``` solution and `DM` is clearly the design matrix.  Add your testing data as well to get a feeling for how good (or bad) the fit on the training data works for it.

In [ ]:
#calculate our betas
betas = ??

#print out our betas
print(betas)


## Part 1d:

Now let's set up to do a fit of our data and add regularization to the mix.  I've provided a starting point to speed you up.  Note that there is a "toggle" to switch between L2 and L1 regularization.  **You'll need to work out the correct regularization loss and gradients for each choice to make the code work properly**.

This step fills a matrix (`weights`) that holds the polynomial weights for a range of regularization strengths, `[rMin,rMax]` in 5 steps.

In the following, you'll be making some plots of these weights. You do not need to turn in plots using both regularization schemes.  It is enough that the plots can be generated as you toggle between L2 and L1 regularization.

In [ ]:
#Linear regression

# Initialize parameters from our fit.  To reduce training time, we may as well
# use results close to our previous step! 
W = betas*0.99

# gradient descent step size, this is a fraction of the gradient 
step_size = 0.1  # Note that if the learning rate is too big, you may not converge!

#Toggle between L2 and L1 regularization
doL2 = 0

# Regularization strength scan parameters
Nsteps = 5  # number of steps for the regularization strength to be tested over
rMin = 0.0  
rMax = 0.1

# this array will hold the values of the regularization strength that we'll test
regRange = np.arange(rMin,rMax,(rMax-rMin)/Nsteps)

# this array will hold the results of our various fits
weights = np.zeros((regRange.size,nDegr+1))

# gradient descent loop
Niter = 10000

# Create a loop for the regularization strength
for ridx, reg in enumerate(regRange):
    #always reset the weights!
    W = betas*0.99
    
    # perform gradient descent with a max of Niter iterations
    for i in range(Niter):
  
        # evaluate function values
        fhat = np.dot(DM,W)

        # compute mean squared loss
        data_loss = np.sum((ytrain-fhat)**2)
        data_loss /= nPts

        # Regularize!
        # L2 & L1 regularization
        # Add the correct regularization loss function
        if doL2 :
            reg_loss = ??
        else:
            reg_loss = ??

        # Total loss is the sum
        loss = reg_loss + data_loss

        # Report progress
        if i % 2000 == 0:
            print("Reg %.3f, iteration %d: loss %f" % (reg,i, loss))
        
        #compute the loss gradients
        dW = -2*np.dot(DM.T,ytrain-fhat)
        dW /= nPts
        
        # Gradient descent needs the regularization gradient
        # You need to add the correct gradients below
        if doL2:
            dW += ?? # regularization gradient for L2
        else:            
            dW += ?? # regularization gradient for L1
        
        # Update weights after this iteration
        # We always correct weights in the negative of the gradient direction,
        # which ensure that we step to the minimum
        W -= step_size*dW

    #capture weights
    weights[ridx,:] = W
    print("Added Weights for Step Index: ", ridx)
    
print(weights)

## Part 1e:

You now have a weights matrix that is of size ```(regRange.size,nDegr+1)```, wherein each row is the weights found for a given regularization strength.  Make two figures using this data:

  1. For each polynomial order, create a line corresponding to the associated coefficient as a function of the regularization strength.  Eg, for polynomial order 5, you will have a line with 10 values corresponding to the 10 steps in the regularization strength.  You'll create a total of 8 lines for polynomial order `[0,7]`.
  2. Plot the resulting function found for each regularization strength value.  Include the testing and training data in the figure.  You will have 10 total lines, one fore each regularization strength.
  
The default maximum regularization strength is 0.1.  Is that a sensible value for both L1 and L2 regularization?  **If not, feel free to edit that value until you get fitted values that feel right to you.**
  
Please use your figures to support answers to the following questions:

  1. What do you conclude about your choice of regularization scheme, L2 vs L1?
  2. At this point, can you state which regularization strength is "best" for each case?


## Part 1f:

Last one!!  To better understand the effect of regularization on this problem, let's study the testing data sample.  Make a final plot that shows the mean squared error loss as a function of regularization strength.  Note that you should be able to reuse a lot of the code from the previous step.  Be careful about finding the mean value, as your testing and training data samples are not the same size!

Use this figure and the previous two to answer the following:

  1. Which regularization strength appears to be best for both L1 and L2 regularization?
  2. For this problem, which regularization scheme appears to work better?  Why?